# StadtRAD Quartale

Dieses Notebook dient dazu, Kelper.gl Visualisierungen für StadtRAD Buchungen aufzuteilen. Der komplette Datensatz wird dabei in Quartale unterteilt. Eine Buchung wird zu einem Quartal gezählt, wenn der Zeitpunkt der **Ausleihe** (`date_from`) in dem entsprechenden Quartal liegt. Pro Quartal wird eine Karte generiert und als HTML-Datei gespeichert, da diese im Browser effektiver angezeigt und bedient werden können.

In [1]:
import json
import pandas as pd
from keplergl import KeplerGl

In [2]:
bookings = pd.read_csv("data/call_a_bike/bookings.csv")
bookings["date_from"] =  pd.to_datetime(bookings["date_from"])
bookings["date_until"] =  pd.to_datetime(bookings["date_until"])

In [3]:
bookings["date_from"].min()

Timestamp('2014-01-01 00:02:51')

In [4]:
bookings["date_until"].max()

Timestamp('2017-05-16 08:14:13')

Zeitspanne: 01.01.2014 - 16.05.2017

Mit dem Toggle `render_map` kann die Darstellung der Karten innerhalb dieses Notebooks aktiviert bzw. deaktiviert werden. Da das Rendern der Karten viele Ressourcen benötigt, sollte das Toggle deaktiviert werden. Die HTML-Karten werden in jedem Fall generiert.

In [5]:
render_map = False

Jede Karte enthält einen Filter, mit dem die Auswahl der Buchungen beschränkt werden kann. Es werden dann nur noch alle Buchungen angezeigt, deren `date_from`  Wert innerhalb des gewählten Zeitfensters liegt. Das Zeitfenster lässt sich nur durch die Anpassung des Schiebereglers verändern. Die Angabe einer konkreten Zahl ist über die Karte leider nicht möglich. Allerdings kann die JSON-Konfiguration angepasst werden. Dafür kann mit dem Wert `date_filter_size_in_h` die Größe des Zeitfensters in Stunden bestimmt werden.

In [6]:
date_filter_size_in_h = 6

In [7]:
def select_data(start_date, end_date):
    mask = (bookings["date_from"] >= start_date) & (bookings["date_from"] <= end_date)
    selected_bookings = bookings.copy().loc[mask]
    selected_bookings["date_from"] = selected_bookings["date_from"].astype(str)
    selected_bookings["date_until"] = selected_bookings["date_until"].astype(str)
    return selected_bookings

In [8]:
def update_time_filter_timestamps(config, start_date):
    filters = config["config"]["visState"]["filters"]
    for filter_config in filters:
        if filter_config["name"] == "date_from":
            timestamp1 = (pd.to_datetime(start_date) - pd.Timestamp("1970-01-01")) // pd.Timedelta("1ms")
            timestamp2 = timestamp1 + (date_filter_size_in_h * 60 * 60)
            filter_config["value"] = [timestamp1, timestamp2]
    return config

In [26]:
def render_map(df, html_file_name):
    with open("config/quarters.json", "r") as file:
        map_config = json.load(file)
    map_config = update_time_filter_timestamps(map_config, df["date_from"].min())
    tmp_map = KeplerGl(height=700, data={"bookings": df}, config=map_config)
    tmp_map.save_to_html(file_name=html_file_name)
    if render_map:
        return tmp_map
    else:
        return None

## Q1 2014

In [27]:
tmp_map = render_map(
    select_data("2014-01-01", "2014-03-31"),
    "generated_maps/2014_q1.html"
)
tmp_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2014_q1.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

Die Konfiguration dieser Map kann gespeichert und für alle weitere übernommen werden, indem das Notebook neu ausgeführt wird.

In [11]:
tmp_map.config

{'version': 'v1',
 'config': {'visState': {'filters': [{'dataId': 'bookings',
     'id': '2l16eicm',
     'name': 'duration_in_min',
     'type': 'range',
     'value': [1, 120],
     'enlarged': False,
     'plotType': 'histogram',
     'yAxis': None},
    {'dataId': 'bookings',
     'id': 'rr5pqlc4k',
     'name': 'date_from',
     'type': 'timeRange',
     'value': [1388534571000, 1388534592600],
     'enlarged': True,
     'plotType': 'histogram',
     'yAxis': None}],
   'layers': [{'id': '6fcgw7',
     'type': 'arc',
     'config': {'dataId': 'bookings',
      'label': 'Buchungen',
      'color': [130, 154, 227],
      'columns': {'lat0': 'from_station_lat',
       'lng0': 'from_station_lng',
       'lat1': 'to_station_lat',
       'lng1': 'to_station_lng'},
      'isVisible': True,
      'visConfig': {'opacity': 0.5,
       'thickness': 1,
       'colorRange': {'name': 'Uber Viz Diverging 3.5',
        'type': 'diverging',
        'category': 'Uber',
        'colors': ['#00939C'

In [12]:
# Aktuelle Konfiguration der Map speichern
if False:
    with open("config/quarters.json", "w") as file:
        json.dump(tmp_map.config, file, indent=2)

## Q2 2014

In [28]:
render_map(
    select_data("2014-04-01", "2014-06-30"),
    "generated_maps/2014_q2.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2014_q2.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q3 2014

In [14]:
render_map(
    select_data("2014-07-01", "2014-09-30"),
    "generated_maps/2014_q3.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2014_q3.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q4 2014

In [15]:
render_map(
    select_data("2014-10-01", "2014-12-31"),
    "generated_maps/2014_q4.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2014_q4.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q1 2015

In [16]:
render_map(
    select_data("2015-01-01", "2015-03-31"),
    "generated_maps/2015_q1.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2015_q1.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q2 2015

In [17]:
render_map(
    select_data("2015-04-01", "2015-06-30"),
    "generated_maps/2015_q2.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2015_q2.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q3 2015

In [18]:
render_map(
    select_data("2015-07-01", "2015-09-30"),
    "generated_maps/2015_q3.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2015_q3.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q4 2015

In [19]:
render_map(
    select_data("2015-10-01", "2015-12-31"),
    "generated_maps/2015_q4.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2015_q4.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q1 2016

In [20]:
render_map(
    select_data("2016-01-01", "2016-03-31"),
    "generated_maps/2016_q1.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2016_q1.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q2 2016

In [21]:
render_map(
    select_data("2016-04-01", "2016-06-30"),
    "generated_maps/2016_q2.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2016_q2.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q3 2016

In [22]:
render_map(
    select_data("2016-07-01", "2016-09-30"),
    "generated_maps/2016_q3.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2016_q3.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q4 2016

In [23]:
render_map(
    select_data("2016-10-01", "2016-12-31"),
    "generated_maps/2016_q4.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2016_q4.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q1 2017

In [24]:
render_map(
    select_data("2017-01-01", "2017-03-31"),
    "generated_maps/2017_q1.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2017_q1.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…

## Q2 2017

In [25]:
render_map(
    select_data("2017-04-01", "2017-06-30"),
    "generated_maps/2017_q2.html"
)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to generated_maps/2017_q2.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'bookings', 'id': '2l16eicm',…